In [1]:
!python -m pip install spiceypy > /dev/null 2>&1

In [2]:
import spiceypy as spy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls > /dev/null 2>&1
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de435.bsp > /dev/null 2>&1
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/gm_de431.tpc > /dev/null 2>&1

In [4]:
spy.furnsh(["naif0012.tls","de435.bsp","gm_de431.tpc"])

#1. Calculate the positions and velocities of the Sun, Jupiter, and Saturn over 100 years in 1-month intervals.

#2.Calculate $K$, $U$ and the total energy of the system at all previous times.

To calculate the total energy of the Solar System, we first need the magnitude of the kinetic energy $(K)$ and the gravitational potential energy $(U)$ of each planet. Thus, to calculate the total energies, we have:

$$K_{total} = \sum_{i=1}^{N} \frac{1}{2}m_i v_i^2 $$

$$U_{total} = \sum_{i\neq j}^{N} -\frac{G m_i m_j}{r^3} r_{ij} $$

Where we can finally obtain the total energy by: $$E_{total} = K_{total} + U_{total}$$

$\textbf{Start Date:}$ April 27, 2001

For this point, we will assume that 1 month = 30 days

In [5]:
# Initial date (your birthday)
et0 = spy.str2et("2001-04-27 00:00:00 UTC-5")

# Time array: 100 years in steps of 1 month (in seconds)
times = np.arange(0, 100 * 365.25 * 86400, 30 * 86400)

# Gravitational constant in km³/kg/s²
G = 6.67430e-20

# Masses from GM / G
bodies = ["Sun", "Jupiter Barycenter", "Saturn Barycenter"]
masses = {body: spy.bodvrd(body, "GM", 1)[1][0] / G for body in bodies}

In [7]:
# Store positions, velocities, energies
positions = {body: [] for body in bodies}
velocities = {body: [] for body in bodies}
K_total, U_total, E_total = [], [], []

for dt in times:
    et = et0 + dt

    # Get state vectors for all bodies at this epoch
    states = {}
    for body in bodies:
        state, _ = spy.spkezr(body, et, "ECLIPJ2000", "None", "SSB")
        r = np.array(state[:3])
        v = np.array(state[3:])
        positions[body].append(r)
        velocities[body].append(v)
        states[body] = {"r": r, "v": v}

    # Compute total kinetic energy
    K = sum(
        0.5 * masses[body] * spy.vnorm(states[body]["v"])**2
        for body in bodies
    )
    K_total.append(K)

    # Compute potential energy from pairwise interactions
    U = 0
    for i in range(len(bodies)):
        for j in range(i + 1, len(bodies)):
            b1, b2 = bodies[i], bodies[j]
            r1, r2 = states[b1]["r"], states[b2]["r"]
            m1, m2 = masses[b1], masses[b2]
            distance = spy.vdist(r1, r2)
            U += -G * m1 * m2 / distance
    U_total.append(U)

    # Compute total energy and convert to Joules (km² → m²)
    E_total.append((K + U) * 1e6)

In [9]:
# Convert energies to Joules
K_total = [K * 1e6 for K in K_total]
U_total = [U * 1e6 for U in U_total]
E_total = [E * 1e6 for E in E_total]

# 3. At the initial time, calculate the forces felt by each body and calculate the value of the virial.

To calculate the forces felt by each body, we use the following formula:

$$ \vec{F_{ij}} = \frac{-G m_i m_j}{r_{ij}^3 } \vec{r_{ij}}$$

We will obtain the virial from: $$V = \sum_{p=1}^{N} \vec{r_p} \cdot \vec{F_p}$$

In [14]:
# Define the initial date
et = spy.str2et("2001-04-27 00:00:00 UTC-5")

# Masses (retrieved using GM / G)
M_sun = spy.bodvrd("Sun", "GM", 1)[1][0] / G
M_jupiter = spy.bodvrd("Jupiter Barycenter", "GM", 1)[1][0] / G
M_saturn = spy.bodvrd("Saturn Barycenter", "GM", 1)[1][0] / G

# Get position vectors of the Sun, Jupiter, and Saturn
state_sun, _ = spy.spkezr("Sun", et, "ECLIPJ2000", "None", "SSB")
state_jupiter, _ = spy.spkezr("Jupiter Barycenter", et, "ECLIPJ2000", "None", "SSB")
state_saturn, _ = spy.spkezr("Saturn Barycenter", et, "ECLIPJ2000", "None", "SSB")

r_sun = np.array(state_sun[:3])
r_jupiter = np.array(state_jupiter[:3])
r_saturn = np.array(state_saturn[:3])

In [12]:
# Compute gravitational forces between each pair
def gravitational_force(m1, m2, r1, r2):
    r_vec = r1 - r2
    r_norm = spy.vnorm(r_vec)
    return -G * m1 * m2 / r_norm**3 * r_vec

In [15]:
# Individual forces
F_sun = gravitational_force(M_sun, M_jupiter, r_sun, r_jupiter) + gravitational_force(M_sun, M_saturn, r_sun, r_saturn)
F_jupiter = gravitational_force(M_jupiter, M_sun, r_jupiter, r_sun) + gravitational_force(M_jupiter, M_saturn, r_jupiter, r_saturn)
F_saturn = gravitational_force(M_saturn, M_sun, r_saturn, r_sun) + gravitational_force(M_saturn, M_jupiter, r_saturn, r_jupiter)

# Compute virial: V = sum(r_i · F_i)
virial = (np.dot(r_sun, F_sun) + np.dot(r_jupiter, F_jupiter) + np.dot(r_saturn, F_saturn)) * 1e6  # to Joules

In [16]:
print("Force on the Sun at initial time:", F_sun, "N")
print("Force on Jupiter at initial time:", F_jupiter, "N")
print("Force on Saturn at initial time:", F_saturn, "N")
print("Virial value at initial time:", virial, "J")
print("Gravitational potential energy at initial time:", U_total[0], "J")

Force on the Sun at initial time: [ 9.90600041e+19  4.64620056e+20 -4.94695058e+18] N
Force on Jupiter at initial time: [-8.06502252e+19 -4.28114876e+20  3.57661650e+18] N
Force on Saturn at initial time: [-1.84097789e+19 -3.65051802e+19  1.37033408e+18] N
Virial value at initial time: -3.869190293804644e+35 J
Gravitational potential energy at initial time: -3.869190293804645e+35 J


The gravitational potential energy \( U \) is a homogeneous function, and according to Euler's theorem for homogeneous functions we have:

$$
\vec{r} \cdot \nabla U = kU = -U
$$

Where \( k \) is the degree of the function, which in this case is \( -1 \) for \( U \).

At the same time, the virial is defined as:

$$
V \equiv \vec{r} \cdot \vec{F}
$$

Thus, we finally obtain:

$$
V = \vec{r} \cdot \vec{F} = \vec{r} \cdot \nabla U (-1) = U
$$

$$
V = U
$$

This is confirmed by our numerical results: both the virial and the total gravitational potential energy of the bodies yielded the same value.

# 4. Average of K, U and E

In [19]:
# Compute averages of kinetic, potential, and total energy
K_avg = np.mean(K_total)
U_avg = np.mean(U_total)
E_avg = np.mean(E_total)

In [20]:
# Print averages
print("Average kinetic energy:", K_avg, "J")
print("Average potential energy:", U_avg, "J")
print("Average total energy:", E_avg, "J")

Average kinetic energy: 1.880751793065654e+35 J
Average potential energy: -3.763834759120145e+35 J
Average total energy: -1.883082966054491e+41 J


In [21]:
# Check virial theorem relations
print("-2 * K_avg =", -2 * K_avg, "J")
print("Initial total energy =", E_total[0], "J")
print("-K_avg =", -K_avg, "J")
print("2 * K_avg + U_avg =", 2 * K_avg + U_avg, "J")

-2 * K_avg = -3.761503586131308e+35 J
Initial total energy = -1.8831331528681246e+41 J
-K_avg = -1.880751793065654e+35 J
2 * K_avg + U_avg = -2.3311729888371826e+32 J


The virial theorem tells us:

$$
\langle U \rangle = -2 \langle K \rangle
$$

$$
E = -\langle K \rangle
$$

We verify that for the Sun–Jupiter–Saturn system, the theorem holds, since the difference between
$$\langle U \rangle$$ and $$-2 \langle K \rangle$$ is less than 1%.

The same applies to the difference between $$E$$ and $$-\langle K \rangle$$.

We finally conclude that this system is a bound system, since the following condition is satisfied:

$$
\dot{G} = 2 \langle K \rangle + \langle U \rangle = 0
$$
